It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values and generating new features from already existing features. 

In our case, you have a few datetime columns to preprocess. you can extract the following from them:

1.weekdays
2.weekends 
3.number of days to holidays
4.Number of days after holiday
5.Beginning of month, mid month and ending of month
(think of more features to extract), extra marks for it


# Preporocessing and feature extraction

In [1]:
import logging
#Logging is a means of tracking events that happen when some software runs
# Create and configure logger
logging.basicConfig(filename="../logs/newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
 
# Test messages
logger.debug("Harmless debug Message")
logger.info("Just an information")
logger.warning("Its a Warning")
logger.error("Did you try to divide by zero")
logger.critical("Internet is down")

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

sns.set_style("darkgrid")
%matplotlib inline
import sys
sys.path.insert(0,'../scripts/')

C:\Users\kachase\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
from data_loader import load_df_from_csv
from data_struct import DataInfo
from cleaner import DataCleaner
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder

C:\Users\kachase\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [4]:

#train = pd.read_csv("../data/clean_train.csv",parse_dates=["Date"], 
                # index_col=["Date"]) # parse the date column (tell pandas column 1 is a datetime)

#test = pd.read_csv("../data/clean_test.csv",parse_dates=["Date"], 
                 #index_col=["Date"])

In [5]:

train = pd.read_csv("../data/clean_train.csv")

test = pd.read_csv("../data/clean_test.csv")

In [6]:
train.head()

,Unnamed: 0,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,...,SalePerCustomer,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,1,3,2013-01-02,5530,668,1,0,0,1,...,8.278443,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,1,4,2013-01-03,4327,578,1,0,0,1,...,7.486159,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,2,1,5,2013-01-04,4486,619,1,0,0,1,...,7.247173,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,3,1,6,2013-01-05,4997,635,1,0,0,1,...,7.869291,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,4,1,1,2013-01-07,7176,785,1,1,0,1,...,9.141401,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [7]:
test.head()

,Unnamed: 0,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Day,...,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,40233,1,6,2015-08-01,1.0,0,0,1,1,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,39377,1,7,2015-08-02,0.0,0,0,1,2,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,2,38521,1,1,2015-08-03,1.0,1,0,1,3,...,32,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,3,37665,1,2,2015-08-04,1.0,1,0,1,4,...,32,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,4,36809,1,3,2015-08-05,1.0,1,0,1,5,...,32,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


## Feature Extraction

### Stores open on during the week

In [8]:
#33 Stores are opened on Weekdays
weekday = train[(train.DayOfWeek != 6) & (train.DayOfWeek != 7)]



In [9]:
weekday.shape

(696693, 25)

### Stores open on Saturday

In [10]:
#33 Stores are opened on Weekdays
Saturday= train[(train.DayOfWeek == 6)]



In [11]:
Saturday.shape

(144052, 25)

### Stores open on Sunday

In [12]:
#33 Stores are opened on Weekdays
Sunday= train[(train.DayOfWeek == 7)]



In [13]:
Sunday.shape

(3593, 25)

### Beginning of month, mid month and ending of month

In [14]:
day_1_month= train[(train.Day == 7)]

In [15]:
# Get total sales, customers and open days per store
store_data_sales = train.groupby([train['Store']])['Sales'].sum()
store_data_customers = train.groupby([train['Store']])['Customers'].sum()
store_data_avg_sales = train.groupby([train['Store']])['Sales'].mean()
store_data_avg_customers = train.groupby([train['Store']])['Customers'].mean()
store_data_open = train.groupby([train['Store']])['Open'].count()

In [16]:
# Sales_per_day, Customers_per_day, avg_sales_per_customer and Sales_per_customers_per_day
# Calculate sales per day, customers per day and sales per customers per day
store_data_sales_per_day = store_data_sales / store_data_open
store_data_customers_per_day = store_data_customers / store_data_open
store_data_avg_sales_per_customer = store_data_avg_sales / store_data_avg_customers
store_data_sales_per_customer_per_day = store_data_sales_per_day / store_data_customers_per_day


In [17]:
#Saving the above values in a dictionary so that they can be mapped to the dataframe.
sales_per_day_dict = dict(store_data_sales_per_day)
customers_per_day_dict = dict(store_data_customers_per_day)
avg_sales_per_customer_dict = dict(store_data_avg_sales_per_customer)
sales_per_customers_per_day_dict = dict(store_data_sales_per_customer_per_day)


In [18]:

train['SalesPerDay'] = train['Store'].map(sales_per_day_dict)
train['Customers_per_day'] = train['Store'].map(customers_per_day_dict)
train['Avg_Sales_per_Customer'] = train['Store'].map(avg_sales_per_customer_dict)
train['Sales_Per_Customers_Per_Day'] = train['Store'].map(sales_per_customers_per_day_dict)

In [19]:


test['Sales_per_day'] = test['Store'].map(sales_per_day_dict)
test['Customers_per_day'] = test['Store'].map(customers_per_day_dict)
test['Avg_Sales_per_Customer'] = test['Store'].map(avg_sales_per_customer_dict)
test['Sales_Per_Customers_Per_Day'] = test['Store'].map(sales_per_customers_per_day_dict)

In [20]:
test.head()

,Unnamed: 0,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Day,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales_per_day,Customers_per_day,Avg_Sales_per_Customer,Sales_Per_Customers_Per_Day
0,0,40233,1,6,2015-08-01,1.0,0,0,1,1,...,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936,8.437366,8.437366
1,1,39377,1,7,2015-08-02,0.0,0,0,1,2,...,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936,8.437366,8.437366
2,2,38521,1,1,2015-08-03,1.0,1,0,1,3,...,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936,8.437366,8.437366
3,3,37665,1,2,2015-08-04,1.0,1,0,1,4,...,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936,8.437366,8.437366
4,4,36809,1,3,2015-08-05,1.0,1,0,1,5,...,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936,8.437366,8.437366


### Holidays

In [21]:
# create holidays dataframe
state_dates = train[(train.StateHoliday == 'a') | (train.StateHoliday == 'b') & (train.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = train[train.SchoolHoliday == 1].loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))  
holidays.sort_values(["ds"], ignore_index=True, inplace=True)
holidays.drop_duplicates(subset ="ds",inplace = True)
holidays.head()

,holiday,ds
0,state_holiday,2013-01-01
34,school_holiday,2013-01-02
1145,school_holiday,2013-01-03
2179,school_holiday,2013-01-04
3212,school_holiday,2013-01-05


In [22]:
holidays.drop('holiday', inplace=True, axis=1)

In [23]:
df = pd.DataFrame()
df["date"] = pd.date_range(start ='1-1-2013',
           end ='31-08-2015', freq ='D')
df.head(5)

,date
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05


In [24]:
df = pd.merge_asof(df, holidays, left_on='date', right_on='ds', direction='forward')
df = pd.merge_asof(df, holidays, left_on='date', right_on='ds')
df.head()

,date,ds_x,ds_y
0,2013-01-01,2013-01-01,2013-01-01
1,2013-01-02,2013-01-02,2013-01-02
2,2013-01-03,2013-01-03,2013-01-03
3,2013-01-04,2013-01-04,2013-01-04
4,2013-01-05,2013-01-05,2013-01-05


In [25]:
df['until'] = df.pop('ds_x').sub(df['date']).dt.days
df['since'] = df['date'].sub(df.pop('ds_y')).dt.days


In [26]:
df.head()

,date,until,since
0,2013-01-01,0.0,0
1,2013-01-02,0.0,0
2,2013-01-03,0.0,0
3,2013-01-04,0.0,0
4,2013-01-05,0.0,0


In [27]:
#state holidays 
train["is_holiday_state"] = train['StateHoliday'].map({"0": 0, "a": 1, "b": 1, "c": 1})

In [28]:
train['Date']=pd.to_datetime(train['Date'])

In [29]:
test['Date']=pd.to_datetime(test['Date'])

In [30]:
# create holidays dataframe
state_dates = train[(train.StateHoliday == 'a') | (train.StateHoliday == 'b') & (train.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = train[train.SchoolHoliday == 1].loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))      
holidays.head()

,holiday,ds
0,state_holiday,2013-10-03
1,state_holiday,2014-10-03
2,state_holiday,2013-05-30
3,state_holiday,2014-06-19
4,state_holiday,2015-06-04


In [31]:
#state holidays 
test["is_holiday_state"] = test['StateHoliday'].map({"0": 0, "a": 1, "b": 1, "c": 1})

In [32]:
#state holidays 
train["is_holiday_state"] = train['StateHoliday'].map({"0": 0, "a": 1, "b": 1, "c": 1})

In [33]:
test.columns

Index(['Unnamed: 0', 'Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Day', 'Month', 'Year', 'DayOfYear',
       'WeekOfYear', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Sales_per_day',
       'Customers_per_day', 'Avg_Sales_per_Customer',
       'Sales_Per_Customers_Per_Day', 'is_holiday_state'],
      dtype='object')

In [34]:

# Remove rows for stores not in test_df
train = train[train["Store"].isin(test.Store.unique())]

In [35]:
train.shape

(641942, 30)

In [36]:
# Save IDs of Closed Stores ("Sales" = 0 assigned later) & Remove Rows
closed_store_ids = test["Id"][test["Open"] == 0].values
test = test[test["Open"] != 0]

In [37]:
# Drop "Open" from test_df
#test.drop(["Open"], axis=1, inplace=True)
'Unnamed: 0'

# Drop "Date" & "YearMonth" column
train.drop(["Date", "Month"], axis=1, inplace=True)
test.drop(["Date", "Month"], axis=1, inplace=True)

In [39]:
train.drop(["Unnamed: 0"], axis=1, inplace=True)

In [40]:
test.drop(["Unnamed: 0"], axis=1, inplace=True)

In [46]:
# Drop "Customers" from training_df
train.drop(['StateHoliday'], axis=1, inplace=True)

In [47]:
# Drop "Customers" from training_df
test.drop(['StateHoliday'], axis=1, inplace=True)

In [48]:
train.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'SchoolHoliday', 'Day',
       'Year', 'DayOfYear', 'WeekOfYear', 'SalePerCustomer', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'SalesPerDay', 'Customers_per_day',
       'Avg_Sales_per_Customer', 'Sales_Per_Customers_Per_Day',
       'is_holiday_state'],
      dtype='object')

In [49]:
test.columns

Index(['Id', 'Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'Day',
       'Year', 'DayOfYear', 'WeekOfYear', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'Sales_per_day',
       'Customers_per_day', 'Avg_Sales_per_Customer',
       'Sales_Per_Customers_Per_Day', 'is_holiday_state'],
      dtype='object')

In [52]:
cleaner = DataCleaner(train)

In [53]:
train_to_process=cleaner.save_clean_data('../data/train_processed.csv')

In [54]:
cleaner = DataCleaner(test)

In [55]:
test_to_process=cleaner.save_clean_data('../data/test_processed.csv')

### Scaling

In [ ]:
train_numerical_features = ['Store', 'DayOfWeek', 'Sales', 'Customers',
       'Open', 'Promo', 'SchoolHoliday', 'Day', 'Month',
       'Year', 'DayOfYear', 'WeekOfYear', 'SalePerCustomer', 'CompetitionDistance',
    'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'SalesPerDay', 'Customers_per_day',
       'Avg_Sales_per_Customer', 'Sales_Per_Customers_Per_Day',
       'is_holiday_state']

In [ ]:
test_numerical_features = ['Store', 'DayOfWeek', 'Open', 'Promo'
        , 'SchoolHoliday', 'Day', 'Month', 'Year', 'DayOfYear',
       'WeekOfYear', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Sales_per_day',
       'Customers_per_day', 'Avg_Sales_per_Customer',
       'Sales_Per_Customers_Per_Day', 'is_holiday_state']

In [ ]:
train_cat_features = ['PromoInterval','Assortment','StoreType']


In [ ]:
test_cat_features = ['PromoInterval','Assortment','StoreType']


In [ ]:
train_features = train.copy()


In [ ]:
test_features = test.copy()


As a final thing, you have to scale the data. This helps with predictions especially when using machine learning algorithms that use Euclidean distances. you can use the standard scaler in sklearn for this.

### Label Encoding

In [ ]:
#casting the columns into strings
labelencoder = LabelEncoder()

#features[cat_features] = features[cat_features].apply(LabelEncoder().fit_transform)
test_features[test_cat_features] = test_features[test_cat_features].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)), axis=0, result_type='expand')

In [ ]:
#casting the columns into strings
labelencoder = LabelEncoder()

#features[cat_features] = features[cat_features].apply(LabelEncoder().fit_transform)
train_features[train_cat_features] = train_features[train_cat_features].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)), axis=0, result_type='expand')

### Scaling

In [ ]:
train_input_features = train_numerical_features + train_cat_features


In [ ]:
test_input_features = test_numerical_features + test_cat_features


In [ ]:
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_features[train_input_features])

In [ ]:
scaler = StandardScaler()
scaled_test = pd.DataFrame(scaler.fit_transform(test_features[test_input_features]))

In [ ]:
scaled_test


Treating each store as an independent regression problem, loop through all stores training the model for the particular store and predicting its sales value. While training, we use logarithm to minimize the likelihood.
Advantages of using logarithm:
- Numerical stability
- Function is smooth and the function is symbolically easier
Features: Promo, SchoolHoliday, Year, Month, DayOfWeek (one-hot encoded), StateHoliday (one-hot encoded), AvgCustStore, AvgCustStoreMonth
Assumptions:
- The store's opening/closing dates does not affect the store's performance. For example, a store that was closed yesterday will not get more sales today because of that.
- The competition of each store will affect it consistently, hence, it does not matter.
- Each store's sales value is independent of the other stores and can be treated as independent regression problems.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf =RandomForestRegressor(n_estimators=300 ,max_depth=8)

In [ ]:
train_dict = dict(list(train.groupby("Store")))
test_dict = dict(list(test.groupby("Store")))
predictions = Series()

for i in test_dict:
    store = training_dict[i]

    X_train = store.drop(["Sales", "Store"], axis=1)
    Y_train = store["Sales"]
    Y_train = np.log(Y_train + 1)
    X_test = test_dict[i].copy()

  

    store_ids = X_test["Id"]
    X_test.drop(["Id", "Store"], axis=1, inplace=True)

    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    randforestreg = RandomForestRegressor()
    randforestreg.fit(X_train, Y_train)
    Y_pred = randforestreg.predict(X_test)

    Y_pred = np.exp(Y_pred) - 1

    predictions = predictions.append(Series(Y_pred, index=store_ids))

predictions = predictions.append(Series(0, index=closed_store_ids))

submission = pd.DataFrame({"Id": predictions.index, "Sales": predictions.values})
submission.to_csv("predictions/randomforestregressor-independent-log.csv", index=False)

In [ ]:
def rmspe(y_true, y_pred):
    diff = y_pred - y_true
    diff_percentage = diff / y_true
    diff_percentage_squared = diff_percentage ** 2
    rmspe = np.sqrt(diff_percentage_squared.mean())
    return rmspe